### Fit selected feature into a Linear Regression Model to make future rental price prediction

In [1]:
import pandas as pd
import glob
import os

path = r'../data/curated/merged_dataset/' # use your path
all_files = glob.glob(os.path.join(path , "*.csv"))

li = []

for filename in sorted(all_files):
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

merged_df = pd.concat(li, axis=0, ignore_index=True)
merged_df.drop(['address', 'latitude', 'longitude', 'postcode', 'sa2_2016'], axis=1, inplace=True)

SELECTED_FEATURES = ['nbed', 'nbath', 'ncar', 'saving_rate', 'min_distance_to_prim', 
'min_distance_to_poli', 'min_distance_to_hosp', 'min_distance_to_cbd', 
'min_distance_to_shop', 'population_density', 'income_per_person', 
'crime_cases', 'min_distance_to_train', 'gdp']

In [20]:
import seaborn as sns


In [2]:
merged_df.rename({'gdp(USD Millioins)': 'gdp', 'saving_rate(% of GDP)': 'saving_rate'}, axis=1, inplace=True)
merged_df = pd.get_dummies(data=merged_df, columns=['sa2_2021'], prefix='sa2')
merged_df = pd.get_dummies(data=merged_df, columns=['residence_type'], prefix='resiType') 
#merged_df.drop(['sa2_2021', 'residence_type'], axis=1, inplace=True)
merged_df.dropna(inplace=True)

y_train = merged_df['weekly_rent']
X_train = merged_df[['year']+SELECTED_FEATURES+list(merged_df.filter(regex='sa2'))+list(merged_df.filter(regex='resiType'))]

In [3]:
from itertools import product
# Construct a prediction dataframe
beds = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]
baths = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]
cars = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
first = list(product(beds, baths, cars))
print(len(first))
first
#internal_dict = {'nbed': beds, 'nbath': baths, 'ncar': cars}
#pd.DataFrame(internal_dict)

640


[(1.0, 1.0, 0),
 (1.0, 1.0, 1),
 (1.0, 1.0, 2),
 (1.0, 1.0, 3),
 (1.0, 1.0, 4),
 (1.0, 1.0, 5),
 (1.0, 1.0, 6),
 (1.0, 1.0, 7),
 (1.0, 1.0, 8),
 (1.0, 1.0, 9),
 (1.0, 2.0, 0),
 (1.0, 2.0, 1),
 (1.0, 2.0, 2),
 (1.0, 2.0, 3),
 (1.0, 2.0, 4),
 (1.0, 2.0, 5),
 (1.0, 2.0, 6),
 (1.0, 2.0, 7),
 (1.0, 2.0, 8),
 (1.0, 2.0, 9),
 (1.0, 3.0, 0),
 (1.0, 3.0, 1),
 (1.0, 3.0, 2),
 (1.0, 3.0, 3),
 (1.0, 3.0, 4),
 (1.0, 3.0, 5),
 (1.0, 3.0, 6),
 (1.0, 3.0, 7),
 (1.0, 3.0, 8),
 (1.0, 3.0, 9),
 (1.0, 4.0, 0),
 (1.0, 4.0, 1),
 (1.0, 4.0, 2),
 (1.0, 4.0, 3),
 (1.0, 4.0, 4),
 (1.0, 4.0, 5),
 (1.0, 4.0, 6),
 (1.0, 4.0, 7),
 (1.0, 4.0, 8),
 (1.0, 4.0, 9),
 (1.0, 5.0, 0),
 (1.0, 5.0, 1),
 (1.0, 5.0, 2),
 (1.0, 5.0, 3),
 (1.0, 5.0, 4),
 (1.0, 5.0, 5),
 (1.0, 5.0, 6),
 (1.0, 5.0, 7),
 (1.0, 5.0, 8),
 (1.0, 5.0, 9),
 (1.0, 6.0, 0),
 (1.0, 6.0, 1),
 (1.0, 6.0, 2),
 (1.0, 6.0, 3),
 (1.0, 6.0, 4),
 (1.0, 6.0, 5),
 (1.0, 6.0, 6),
 (1.0, 6.0, 7),
 (1.0, 6.0, 8),
 (1.0, 6.0, 9),
 (1.0, 7.0, 0),
 (1.0, 7.0, 1),
 (1.0, 7

In [4]:
df1,df2,df3 = [pd.DataFrame({name:lst} ) for lst,name in zip([beds,baths,cars], ['nbed', 'nbath', 'ncar'])]

df_123 = df1.join(df2, how='cross').join(df3, how='cross')

internal_df = df_123.query("nbed >= nbath >= ncar").reset_index(drop=True)
internal_df

,nbed,nbath,ncar
0,1.0,1.0,0
1,1.0,1.0,1
2,2.0,1.0,0
3,2.0,1.0,1
4,2.0,2.0,0
...,...,...,...
151,8.0,8.0,4
152,8.0,8.0,5
153,8.0,8.0,6
154,8.0,8.0,7
